In [2]:
import twitter
import pandas as pd

from collections import defaultdict
import re
import string
import time
import os
import requests
import bs4
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Twiiter API

In [2]:
def getTwitterAPI_data(keyword,tweet_count):
    CONSUMER_KEY = "rYIrIgoJaKS8A5YqCAAVtbkz9"  
    CONSUMER_SECRET = "wmRnDK0QCOSt3k8JsDTDAYoClSRr0IUTTTNZDd7PkEVpJ1FaHd"  
    ACCESS_TOKEN_KEY = "1269476398042738688-KspldUGyYSoB1GwCk0yx8TLnjTMC45"  
    ACCESS_TOKEN_SECRET = "KoUvNsJpGBcvj7gR1YfYFLMgJRuWD3n87wBDYOW00SdLk"  
    api = twitter.Api(  
        consumer_key=CONSUMER_KEY,  
        consumer_secret=CONSUMER_SECRET,  
        access_token_key=ACCESS_TOKEN_KEY,  
        access_token_secret=ACCESS_TOKEN_SECRET,  
    )  
    # The search term you want to find
    query = keyword
    # Language code (follows ISO 639-1 standards)
    language = "en"

    # Calling the user_timeline function with our parameters
    results = api.GetSearch(term=query, lang=language,count=tweet_count)

    return results

# Twiiter Data from GitHub

In [3]:
def load_Twitter_GitHub_data(file_name):
    tweet_data = pd.read_csv(file_name)
    return tweet_data

In [4]:
def parseText(text):
    text = text.replace(u'\xa0', u' ')
    output=re.findall('>([^<]+)<',text)
    final_output="".join(output)
    return final_output

# Webscrapping on FOX News 

In [5]:
def get_FOX_news(chrome_driver_path,base_url,n_news):
    browser = webdriver.Chrome(chrome_driver_path)
    browser.get(base_url)
    time.sleep(2)
    n_page = int(n_news/10)-1
    for i in range(0,n_page):
        webElem5 = browser.find_element_by_xpath('//*/div[@class="button load-more js-load-more"]')
        webElem5.click()
        time.sleep(2)
    elem1 = browser.find_element_by_xpath("//*/div[@class = 'content article-list']")
    links = elem1.find_elements_by_tag_name("a")
    desired_links =[]
    for link in links:
        hyperlink = link.get_attribute("href")
        if hyperlink not in desired_links and "video" not in hyperlink:
            desired_links.append(hyperlink)
    browser.close()
    browser.quit()
    article_list = []
    category_list = []
    published_date_list =[]
    title_list = []
    for url in desired_links:
        html_page = requests.get(url)
        hp = html_page.text
        soup = bs4.BeautifulSoup(hp)
        category =url.split("/")[3]
        category_list.append(category)
        meta2 = soup.select('meta[name="dc.date"]')
        published_date = meta2[0].get("content")
        published_date_list.append(published_date)
        meta3 = soup.select('meta[name="dc.title"]')
        title = meta3[0].get("content")
        title_list.append(title)
        paragraph =soup.find_all("p")
        article_content = []
        for para in paragraph:
            para_class = para.get("class")
            text =parseText(str(para))
            if para_class is None:
                if text !="" and "FOX" not in text.upper():
                    article_content.append(text)
            elif para_class[0] == 'speakable' and "FOX" not in text.upper():
                if text !="":
                    article_content.append(text)
        article_full = "\n".join(article_content)
        article_list.append(article_full)
    merged_data ={"Published_Date" :published_date_list,
             "News Source":["Fox News"]*len(article_list),
             "URL":desired_links,
             "Title":title_list,
             "Category":category_list,
             "Content":article_list}
    merged_df= pd.DataFrame(merged_data)
    return merged_df

# Webscrapping on NY Times

In [6]:
def get_NYTimes_news(chrome_driver_path,base_url,n_news):
    browser = webdriver.Chrome(chrome_driver_path)
    browser.get(base_url)
    time.sleep(2)
    n_page = int(n_news/10)-1
    for i in range(0,n_page):
        webElem5 = browser.find_element_by_xpath('//*/button[@data-testid="search-show-more-button"]')
        webElem5.click()
        time.sleep(2)
    elem1 = browser.find_elements_by_xpath("//*/li[@data-testid = 'search-bodega-result']")
    desired_links =[]
    for elem in elem1:
        link = elem.find_element_by_tag_name("a")
        hyperlink = link.get_attribute("href")
        desired_links.append(hyperlink)
    browser.close()
    browser.quit()
    article_list = []
    category_list = []
    published_date_list =[]
    title_list = []
    for url in desired_links:
        html_page = requests.get(url)
        hp = html_page.text
        soup = bs4.BeautifulSoup(hp)
        meta1 = soup.select('meta[name="CG"]')
        category = meta1[0].get("content")
        category_list.append(category)
        meta2 = soup.select('meta[name="pdate"]')
        text = meta2[0].get("content")
        published_date =text[0:4]+"-"+text[4:6]+"-"+text[6:8]
        published_date_list.append(published_date)
        meta3 = soup.select('meta[property="og:title"]')
        title = meta3[0].get("content")
        title_list.append(title)
        paragraph =soup.find_all("p")
        article_content = []
        for para in paragraph:
            para_class = para.get("class")
            text =parseText(str(para))
            if para.string not in ['Advertisement','Supported by'] and text !="":
                article_content.append(text)
        article_full = "\n".join(article_content)
        article_list.append(article_full)
    merged_data ={"Published_Date" :published_date_list,
         "News Source":["NY Times"]*len(article_list),
         "URL":desired_links,
         "Title":title_list,
         "Category":category_list,
         "Content":article_list}
    merged_df= pd.DataFrame(merged_data)
    return merged_df

# Webscrapping on CNN news

In [1]:
def get_CNN_news(chrome_driver_path,base_url,n_news):
    browser = webdriver.Chrome(chrome_driver_path)
    browser.get(base_url)
    time.sleep(2)
    desired_links =[]
    i=1
    while len(desired_links)<n_news:
        url= 'https://edition.cnn.com/search?q=coronavirus&size=10&from='+str((i-1)*10)+'&page='+str(i)
        i=i+1
        browser.get(url)
        time.sleep(2)
        elem1 = browser.find_elements_by_xpath("//*/div[@class = 'cnn-search__result cnn-search__result--article']")
        for elem in elem1:
            link = elem.find_element_by_tag_name("a")
            hyperlink = link.get_attribute("href")
            if len(desired_links) <n_news:
                desired_links.append(hyperlink)
    browser.close()
    browser.quit()
    article_list = []
    category_list = []
    published_date_list =[]
    title_list = []
    for url in desired_links:
        html_page = requests.get(url)
        hp = html_page.text
        soup = bs4.BeautifulSoup(hp)
        meta1 = soup.select('meta[name="section"]')
        if len(meta1)==0:
            meta1 = soup.select('meta[name="meta-section"]')
        category = meta1[0].get("content")
        category_list.append(category)
        meta2 = soup.select('meta[name="pubdate"]')
        if len(meta2)==0:
            meta2 = soup.select('meta[property="article:published_time"]')
        text = meta2[0].get("content")
        published_date =text[0:10]
        published_date_list.append(published_date)
        meta3 = soup.select('meta[property="og:title"]')
        title = meta3[0].get("content")
        title_list.append(title)
        paragraph =soup.find_all("div",{'class':'zn-body__paragraph'})
        article_content = []
        for para in paragraph:
            text =parseText(str(para))
            if text !="":
                article_content.append(text)
        article_full = "\n".join(article_content)
        article_list.append(article_full)
    merged_data ={"Published_Date" :published_date_list,
             "News Source":["CNN"]*len(article_list),
             "URL":desired_links,
             "Title":title_list,
             "Category":category_list,
             "Content":article_list}
    merged_df= pd.DataFrame(merged_data)
    return merged_df

In [8]:
def save_news_data(article_list,folder_name,file_prefix):
    i=1
    for article in article_list:
        f = open(folder_name+"/"+file_prefix+"_"+str(i)+".txt","w",encoding="utf-8") 
        f.write(article)
        f.close()
        i+=1 
    return True

In [9]:
def save_news_data2(article_list,folder_name,file_prefix):
    i=1
    for article in article_list:
        f = open(folder_name+"/"+file_prefix+"_"+str(i)+".txt","w",encoding="utf-8") 
        f.write(article)
        f.close()
        i+=1 
    return True

In [10]:
def load_news_data(folder_name,file_prefix):
    article_list=[]
    for filename in os.listdir(folder_name):
        if filename.startswith(file_prefix+"_"):
            f = open(folder_name+"/"+filename,"r",encoding="utf-8") 
            article = f.read()
            article_list.append(article)
    return article_list

In [11]:
#chrome_driver_path = r'C:\Users\admin\Downloads\chromedriver_win32\chromedriver.exe'
#base_url = 'https://www.foxnews.com/category/health/infectious-disease/coronavirus'
#n_news = 100
#data =get_FOX_news(chrome_driver_path,base_url,n_news)

In [13]:
#chrome_driver_path = r'C:\Users\admin\Downloads\chromedriver_win32\chromedriver.exe'
#base_url = 'https://www.nytimes.com/search?query=coronavirus'
#n_news = 100
#data =get_NYTimes_news(chrome_driver_path,base_url,n_news)

In [4]:
#chrome_driver_path = r'C:\Users\admin\Downloads\chromedriver_win32\chromedriver.exe'
#base_url = 'https://edition.cnn.com/search?q=coronavirus&size=10&from=0&page=1'
#n_news = 100
#data =get_CNN_news(chrome_driver_path,base_url,n_news)

In [17]:
#data.to_pickle("test.pkl")

In [18]:
#data2 =pd.read_pickle("test.pkl")

In [19]:
#data2

,Published_Date,News Source,URL,Title,Category,Content
0,2020-08-23,CNN,https://www.cnn.com/world/live-news/coronaviru...,Coronavirus update: Latest news from around th...,world,
1,2020-08-22,CNN,https://www.cnn.com/world/live-news/coronaviru...,August 22 coronavirus news,world,
2,2020-08-23,CNN,https://www.cnn.com/2020/08/23/us/california-w...,Nearly 1 million acres are burning due to wild...,us,"Hundreds of fires were started by lightning, C..."
3,2020-08-23,CNN,https://www.cnn.com/2020/08/23/tech/algorithms...,Algorithms promised efficiency. But they've wo...,tech,"The 18-year-old, whose full name CNN is not di..."
4,2020-08-22,CNN,https://www.cnn.com/2020/08/22/health/us-coron...,"Nearly 70,000 lives could be saved in the next...",health,The projection by the University of Washington...
...,...,...,...,...,...,...
95,2020-08-22,CNN,https://www.cnn.com/2020/08/22/us/california-w...,California fires: Firefighters say they're str...,us,"Hundreds of active fires, including more than ..."
96,2020-08-23,CNN,https://www.cnn.com/2020/08/22/asia/chinas-com...,China's Communist Party is a threat to the wor...,asia,"More recently, she caused a stir with a spate ..."
97,2020-08-22,CNN,https://www.cnn.com/2020/08/22/opinions/covid-...,Opinion: The sucker punch to America's Covid f...,opinions,After three months of a relentless cycle of br...
98,2020-08-22,CNN,https://www.cnn.com/2020/08/22/politics/postal...,House approves bill to send $25 billion to Pos...,politics,"The bill passed 257-150, largely along party l..."
